In [1]:
import pandas as pd
import psycopg2
from datetime import date, timedelta
from IPython.core.display import HTML, display

In [74]:
def link_to(url, text):
    return f"""<a target="_blank" href="{url}">{text}</a>"""

def print_df(df):
    display(HTML(df.to_html(escape=False)))
    
def get_largest_eth_deposits(exchange, start_day, end_day, n=10):
    with psycopg2.connect("postgresql://postgres@localhost:7432/postgres-factory-staging") as conn:
        with conn.cursor() as cursor:
            cursor.execute(f"""
                WITH addrs AS (
                    SELECT address 
                    FROM tagged_addresses
                    WHERE blockchain = 'ETHEREUM'
                    AND LOWER(owner) = '{exchange.lower()}'
                ), tagged AS (
                    SELECT address 
                    FROM tagged_addresses
                    WHERE blockchain = 'ETHEREUM'
                )
                SELECT time, source, hash, value
                FROM transfers
                WHERE asset = 'ETH' 
                AND destination IN (SELECT address FROM addrs)
                AND source NOT IN (SELECT address FROM tagged)
                AND time >= '{start_day.isoformat()}'
                AND time <= '{end_day.isoformat()}'
                ORDER BY value DESC
                LIMIT {n}
            """)
            
            df = pd.DataFrame(data=cursor.fetchall(), columns=["time", "address", "tx", "value"])
            df["value"] = df["value"].apply(lambda x: round(x, 2))
            df["link"] = df["tx"].apply(lambda x: link_to(f"https://etherscan.com/tx/0x{x}", "link"))
            del df["tx"]
            return df

def get_largest_eth_withdrawals(exchange, start_day, end_day, n=10):
    with psycopg2.connect("postgresql://postgres@localhost:7432/postgres-factory-staging") as conn:
        with conn.cursor() as cursor:
            cursor.execute(f"""
                WITH addrs AS (
                    SELECT address 
                    FROM tagged_addresses
                    WHERE blockchain = 'ETHEREUM'
                    AND LOWER(owner) = '{exchange.lower()}'
                ), tagged AS (
                    SELECT address 
                    FROM tagged_addresses
                    WHERE blockchain = 'ETHEREUM'
                )
                SELECT time, destination, hash, value
                FROM transfers
                WHERE asset = 'ETH' 
                AND source IN (SELECT address FROM addrs)
                AND destination NOT IN (SELECT address FROM tagged)
                AND time >= '{start_day.isoformat()}'
                AND time <= '{end_day.isoformat()}'
                ORDER BY value DESC
                LIMIT {n}
            """)
            
            df = pd.DataFrame(data=cursor.fetchall(), columns=["time", "address", "tx", "value"])
            df["value"] = df["value"].apply(lambda x: round(x, 2))
            df["link"] = df["tx"].apply(lambda x: link_to(f"https://etherscan.com/tx/0x{x}", "link"))
            del df["tx"]
            return df
        
def get_tagged_addresses(exchange, chain):
    with psycopg2.connect("postgresql://postgres@localhost:7432/postgres-factory-staging") as conn:
        with conn.cursor() as cursor:
            cursor.execute(f"""
                SELECT address
                FROM tagged_addresses
                WHERE owner = '{exchange}'
                AND blockchain = '{chain}'
            """)
            
            return {row[0] for row in cursor.fetchall()}
        
    
# Transforms 6442168329254700655333774564790058229716727912617541056095553257796124352664
# into 0e3e2357e806b6cdb1f70b54c3a3a17b6714ee1f0e68bebb44a74b1efd512098
def int_to_hash(i):
    return "%064x" % int(i)

def get_largest_btc_deposits(exchange, start_day, end_day, n=10):
    # First collect tagged addresses for that exchange
    tagged_addresses = get_tagged_addresses(exchange, "BITCOIN")
        
    with psycopg2.connect("postgresql://postgres@localhost:7432/postgres4") as conn:
        with conn.cursor() as cursor:
            cursor.execute(f"""
                SELECT output_median_time_created, output_addresses[1], output_tx_hash, output_value_satoshi / 100000000.0
                FROM btc_outputs
                WHERE output_median_time_created >= '{start_day.isoformat()}'
                AND output_median_time_created <= '{end_day.isoformat()}'
            """)
            
            data = cursor.fetchall()
            data = [x for x in data if x[1] in tagged_addresses]
            
            df = pd.DataFrame(data=data, columns=["time", "address", "tx", "value"])
            df["value"] = df["value"].apply(lambda x: round(x, 2))
            df["link"] = df["tx"].apply(lambda x: link_to(f"https://btc.com/tx/{int_to_hash(x)}", "link"))
            del df["tx"]
            return df.sort_values(by=["value"], ascending=False).head(n)
        
def get_largest_btc_withdrawals(exchange, start_day, end_day, n=10):
    # First collect tagged addresses for that exchange
    tagged_addresses = get_tagged_addresses(exchange, "BITCOIN")
        
    with psycopg2.connect("postgresql://postgres@localhost:7432/postgres4") as conn:
        with conn.cursor() as cursor:
            cursor.execute(f"""
                SELECT output_median_time_spent, output_addresses[1], output_spending_tx_hash, output_value_satoshi / 100000000.0
                FROM btc_outputs
                WHERE output_median_time_spent >= '{start_day.isoformat()}'
                AND output_median_time_spent <= '{end_day.isoformat()}'
            """)
            
            data = cursor.fetchall()
            # Then filter outputs based on whether the output address belongs to the exchange
            data = [x for x in data if x[1] in tagged_addresses]
            
            df = pd.DataFrame(data=data, columns=["time", "address", "tx", "value"])
            df["value"] = df["value"].apply(lambda x: round(x, 2))
            df["link"] = df["tx"].apply(lambda x: link_to(f"https://btc.com/tx/{int_to_hash(x)}", "link"))
            del df["tx"]
            return df.sort_values(by=["value"], ascending=False).head(n)

In [81]:
start = date(2017, 6, 17)
end = start + timedelta(days=4)

df = get_largest_btc_deposits("Kraken", start, end, n=25)
print_df(df)

,time,address,value,link
538,2017-06-19 19:58:23,17QVYyUMgWzZaTBHgXeA5hM5tnWHXtEmnQ,478.00,link
833,2017-06-20 15:47:22,1CwaaeLiHqsbYszygi1vhbv6atLMPE4hJo,300.00,link
713,2017-06-20 10:46:31,19Wa69eDP1F6J2Segjsd8co7bVjdLFeL2n,150.00,link
858,2017-06-20 18:35:52,1Kd6zLb9iAjcrgq8HzWnoWNVLYYWjp3swA,146.04,link
634,2017-06-20 05:27:46,1Kd6zLb9iAjcrgq8HzWnoWNVLYYWjp3swA,49.51,link
491,2017-06-19 15:37:14,1Kd6zLb9iAjcrgq8HzWnoWNVLYYWjp3swA,49.38,link
909,2017-06-20 20:32:05,1Kd6zLb9iAjcrgq8HzWnoWNVLYYWjp3swA,48.72,link
901,2017-06-20 20:32:05,1Kd6zLb9iAjcrgq8HzWnoWNVLYYWjp3swA,48.71,link
905,2017-06-20 20:32:05,1Kd6zLb9iAjcrgq8HzWnoWNVLYYWjp3swA,48.71,link
904,2017-06-20 20:32:05,1Kd6zLb9iAjcrgq8HzWnoWNVLYYWjp3swA,48.71,link
